In [2]:
from datasets import load_dataset
import pandas as pd

In [5]:
dataset = load_dataset("Atipico1/incontext_tqa_v2", split="train")
dataset = dataset.remove_columns(["answerable", "hasanswer"])
dataset.push_to_hub("Atipico1/incontext_tqa")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/12 [00:00<?, ?ba/s]

In [ ]:
dataset["ctxs"][0]

In [ ]:
uncertain       1694
answerable      1433
unanswerable     483

In [ ]:
answerable
uncertain       2046
answerable      1433
unanswerable     131

In [5]:
df = pd.DataFrame(dataset)
df.answerable.value_counts()

answerable
uncertain       2238
answerable       889
unanswerable     483
Name: count, dtype: int64

In [12]:
def hasanswer(ctxs) -> bool:
    return any([c["hasanswer"] for c in ctxs])

def answerable(ctxs) -> bool:
    res = []
    for ctx in ctxs:
        hasanswer, entail = ctx["hasanswer"], ctx["nli"]
        if hasanswer and (entail in ["entailment", "contradiction"]):
            res.append("answerable")
        elif (not hasanswer) and (entail != "entailment"):
            res.append("unanswerable")
        else:
            res.append("uncertain")
    if res.count("answerable") >= 1:
        return "answerable"
    elif res.count("unanswerable") == 5:
        return "unanswerable"
    else:
        return "uncertain"

In [10]:
dataset = dataset.map(lambda x: {"hasanswer": hasanswer(x["ctxs"])})
dataset = dataset.map(lambda x: {"answerable": answerable(x["ctxs"])})

Map:   0%|          | 0/3610 [00:00<?, ? examples/s]

Map:   0%|          | 0/3610 [00:00<?, ? examples/s]

In [11]:
df = pd.DataFrame(dataset)
df.answerable.value_counts()

answerable
uncertain       1694
answerable      1433
unanswerable     483
Name: count, dtype: int64